In [1]:
import errno
import glob
import json
import os
import re
import shutil
from types import SimpleNamespace
import cv2
import torch
import warnings
from lightning_objects import LightningModel
warnings.filterwarnings('ignore')
from config import Configuration
import pandas as pd
from common_utils import stratify_split, make_holdout_df, set_seeds
from train_manager import TrainManager

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def main(experiment_name: str, debug, resume=False,
         finetune=False, freeze_bn=True, freeze_feature_extractor=False):

    experiment_dir = os.path.abspath(f'trained-models/{experiment_name}')
    print('Experiment directory', experiment_dir)

    try:
        # -------- SETUP --------
        checkpoint_params = None
        finetune_model_fnames = None
        folds_df, holdout_df = None, None

        if not resume and not finetune: # totally new experiment
            make_experiment_directory(experiment_dir)
            config = Configuration()
            config.debug = debug
            set_seeds(config.seed)

            # -------- LOAD DATA FROM TRAIN FILE --------
            data_df = pd.read_csv(config.data_dir + '/train.csv', engine='python')
            data_df, holdout_df = make_holdout_df(data_df, seed=config.seed)
            folds_df = stratify_split(data_df, config.fold_num, config.seed, config.target_col)

            # -------- SAVE FILES (for experiment state) --------
            folds_df.to_csv(experiment_dir + '/folds.csv', index=False)
            # save holdout to a csv file for final inference (so we don't run inference on training examples)
            holdout_df.to_csv(experiment_dir + '/holdout.csv', index=False)
            with open(experiment_dir + '/experiment_config.json', 'w') as f:
                json.dump(config.__dict__, f)
        elif resume or finetune:
            # LOAD DATA FROM SAVED FILES
            with open(experiment_dir + '/experiment_config.json', 'r') as f:
                config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
                set_seeds(config.seed)
                config.debug = debug

            folds_df = pd.read_csv(experiment_dir + '/folds.csv', engine='python')
            holdout_df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')

            if finetune and not resume:
                print('finetuning...')
                # verify there are checkpoints to fine tune
                finetune_model_fnames = glob.glob(experiment_dir + '/*fold*.ckpt')
                assert len(finetune_model_fnames) > 0
                finetune_model_fnames.sort()

                # make new directory for tuning experiment with files from training run 1
                make_experiment_directory(experiment_dir + '_tune')
                for f in os.listdir(experiment_dir):
                    print(f"copying {f} to {experiment_dir + '_tune'}")
                    shutil.copy2(experiment_dir + '/' + f, experiment_dir + '_tune')
                experiment_dir += '_tune'
                experiment_name += '_tune'
            else:
                print('resuming from last checkpoint...')
                checkpoint_params = get_checkpoint_params(experiment_dir, resume, config.model_arch)

        assert holdout_df is not None, 'holdout_df is None'
        assert folds_df is not None, 'folds_df is None'

        # cv2 multithreading seems to go into deadlock with PyTorch data loaders
        if config.num_workers > 0:
            cv2.setNumThreads(0)

        trainer = TrainManager(experiment_name=experiment_name, experiment_dir=experiment_dir,
                               folds_df=folds_df, holdout_df=holdout_df,
                               checkpoint_params=checkpoint_params, config=config,
                               finetune=finetune, freeze_bn=freeze_bn,
                               freeze_feature_extractor=freeze_feature_extractor,
                               finetune_model_fnames=finetune_model_fnames)
        trainer.run()
    finally:
        torch.cuda.empty_cache()

def make_experiment_directory(name):
    try:
        os.makedirs(name)
    except FileExistsError as e:
        print('Experiment already exists. Be sure to resume training appropriately or start a new experiment.')
        if e.errno == errno.EEXIST: raise


def get_checkpoint_params(basename, resume, model_arch):
    checkpoint_params = None
    if resume:
        checkpoint_params = {}
        model_filenames = glob.glob(basename + '/*fold*.ckpt')
        trained_folds = [re.findall(r'fold\d+', f)[0][len('fold'):] for f in model_filenames]
        most_recent_fold = int(max(trained_folds)) if len(trained_folds) > 0 else 0

        checkpoint_params['restart_from'] = most_recent_fold
        checkpoint_params['checkpoint_file_path'] = f'{basename}/{model_arch}_fold{most_recent_fold}.pth'
        #checkpoint_params['checkpoint_file_path'] = f'{basename}/{model_arch}_fold{1}.pth'

    return checkpoint_params

In [ ]:
if __name__ == '__main__':
    try:
        debug = False
        print('Running in debug mode:', debug)
        main(experiment_name='adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0', debug=debug,
             resume=False, finetune=False, freeze_bn=True, freeze_feature_extractor=False)
    except KeyboardInterrupt:
        pass

Running in debug mode: False
Experiment directory /opt/favordata/AI/Felix/kaggle-cassava/trained-models/adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0
folds_df len 18187, holdout_df len 3210
Training fold 0
Class sample counts [ 758 1470 1623 8933 1765]
After class sample counts [2274 2940 3732 8933 4765]


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test_accuracy  | Accuracy       | 0     
2 | criterion      | BiTemperedLoss | 0     
3 | model          | EfficientNet   | 17.6 M
--------------------------------------------------
17.4 M    Trainable params
125 K     Non-trainable params
17.6 M    Total params
Finding best initial lr: 100%|██████████| 100/100 [01:57<00:00,  1.16s/it]Restored states from the checkpoint file at /opt/favordata/AI/Felix/kaggle-cassava/trained-models/adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0/lr_find_temp_model.ckpt
Learning rate set to 0.0019054607179632484

  | Name           | Type           | Params
--------------------------------------------------
0 | valid_accuracy | Accuracy       | 0     
1 | test

Epoch 1:  47%|████▋     | 451/967 [02:16<22:33,  2.62s/it, loss=0.202, v_num=1, val_loss=0.487, val_acc=0.258, train_loss=0.233]    

Finding best initial lr: 100%|██████████| 100/100 [04:17<00:00,  2.58s/it]

Epoch 1:  47%|████▋     | 452/967 [02:16<22:09,  2.58s/it, loss=0.203, v_num=1, val_loss=0.487, val_acc=0.258, train_loss=0.164]

...

Epoch 13: 100%|██████████| 967/967 [26:35<00:00,  2.81s/it, loss=0.133, v_num=3, val_loss=0.11, val_acc=0.881, train_loss=0.128]

Finding best initial lr: 100%|██████████| 100/100 [21:36<00:00, 12.97s/it]

Epoch 5:  94%|█████████▍| 910/967 [04:35<00:30,  1.86it/s, loss=0.157, v_num=4, val_loss=0.111, val_acc=0.876, train_loss=0.123]     
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 57/57 [00:18<00:00,  3.83it/s]

Epoch 5, step 1140: val_loss was not in top 1


Epoch 6:  90%|████████▉ | 869/967 [04:23<00:54,  1.79it/s, loss=0.154, v_num=4, val_loss=0.115, val_acc=0.867, train_loss=0.0587]    